# T1137 - Office Application Startup
Adversaries may leverage Microsoft Office-based applications for persistence between startups. Microsoft Office is a fairly common application suite on Windows-based operating systems within an enterprise network. There are multiple mechanisms that can be used with Office for persistence when an Office-based application is started; this can include the use of Office Template Macros and add-ins.

A variety of features have been discovered in Outlook that can be abused to obtain persistence, such as Outlook rules, forms, and Home Page.(Citation: SensePost Ruler GitHub) These persistence mechanisms can work within Outlook or be used through Office 365.(Citation: TechNet O365 Outlook Rules)

## Atomic Tests

### Atomic Test #1 - Office Application Startup - Outlook as a C2As outlined in MDSEC's Blog post https://www.mdsec.co.uk/2020/11/a-fresh-outlook-on-mail-based-persistence/ 
it is possible to use Outlook Macro as a way to achieve persistance and execute arbitrary commands. This transform Outlook into a C2.
Too achieve this two things must happened on the syste
- The macro security registry value must be set to '4'
- A file called VbaProject.OTM must be created in the Outlook Folder.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Office\16.0\Outlook\Security\Level" /t REG_DWORD /d 1 /f
mkdir  %APPDATA%\Microsoft\Outlook\ >nul 2>&1
echo "Atomic Red Team TEST" > %APPDATA%\Microsoft\Outlook\VbaProject.OTM
```

In [ ]:
Invoke-AtomicTest T1137 -TestNumbers 1

#### Cleanup: 
```cmd
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Office\16.0\Outlook\Security\Level" /f >nul 2>&1
del %APPDATA%\Microsoft\Outlook\VbaProject.OTM >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1137 -TestNumbers 1 -Cleanup

## Detection
Collect process execution information including process IDs (PID) and parent process IDs (PPID) and look for abnormal chains of activity resulting from Office processes. Non-standard process execution trees may also indicate suspicious or malicious behavior. If winword.exe is the parent process for suspicious processes and activity relating to other adversarial techniques, then it could indicate that the application was used maliciously.

Many Office-related persistence mechanisms require changes to the Registry and for binaries, files, or scripts to be written to disk or existing files modified to include malicious scripts. Collect events related to Registry key creation and modification for keys that could be used for Office-based persistence.(Citation: CrowdStrike Outlook Forms)(Citation: Outlook Today Home Page)

Microsoft has released a PowerShell script to safely gather mail forwarding rules and custom forms in your mail environment as well as steps to interpret the output.(Citation: Microsoft Detect Outlook Forms) SensePost, whose tool [Ruler](https://attack.mitre.org/software/S0358) can be used to carry out malicious rules, forms, and Home Page attacks, has released a tool to detect Ruler usage.(Citation: SensePost NotRuler)


## Shield Active Defense
### System Activity Monitoring 
 Collect system activity logs which can reveal adversary activity. 

 Capturing system logs can show logins, user and system events, etc.  Collecting this data and potentially sending it to a centralized location can help reveal the presence of an adversary and the actions they perform on a compromised system.
#### Opportunity
There is an opportunity to create a detection with a moderately high probability of success.
#### Use Case
A defender can collect system process information and look for abnormal activity tied to Office processes.
#### Procedures
Ensure that systems capture and retain common system level activity artifacts that might be produced.
Monitor Windows systems for event codes that reflect an adversary changing passwords, adding accounts to groups, etc.
